In [1]:
import transformers
transformers.logging.set_verbosity_error()

In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
dev_df = pd.read_csv('data/input/dev.tsv', sep='\t')
toxic_inputs = dev_df['toxic_comment'].tolist()

## Library version

In [1]:
from russian_paraphrasers import GPTParaphraser

In [2]:
model_name = 'russian-paraphrasers-gpt3'

In [6]:
paraphraser = GPTParaphraser(model_name="gpt3", range_cand=True, make_eval=False)

In [7]:
dev_df = pd.read_csv('data/input/dev.tsv', sep='\t')
toxic_inputs = dev_df['toxic_comment'].tolist()

In [8]:
sentence = "Мерседес это самая быстрая машина на российском рынке."

In [9]:
results = paraphraser.generate(
    sentence, n=10, temperature=1, 
    top_k=10, top_p=0.9, 
    max_length=100, repetition_penalty=1.5,
    threshold=0.7
)
results

{'origin': 'Мерседес это самая быстрая машина на российском рынке.',
 'results': [{'predictions': ['Merce признали самой быстрой машиной в России.',
    'Самая быстрорастущая компания по производству авто в России.',
    'Самая быстрорастущая марка автомобилей в России.',
    'Merce является самой быстрорастущей маркой автомобилей в России.',
    'Мерседесе назвали самой быстрорастущей маркой в России.',
    'Самая быстрорастущая марка автомобилей в России.',
    'Мерседесе признали самой быстрой машиной в России.',
    'Самая скоростная российская марка авто "мерседесе".',
    'Названа максимальная скорость самой быстрой иномарки в России.',
    'Названа наиболее быстрорастущая марка легковых автомобилей в России.'],
   'best_candidates': ['Самая быстрорастущая компания по производству авто в России.',
    'Мерседесе назвали самой быстрорастущей маркой в России.',
    'Названа наиболее быстрорастущая марка легковых автомобилей в России.',
    'Самая быстрорастущая марка автомобилей в 

In [10]:
results['results']

[{'predictions': ['Merce признали самой быстрой машиной в России.',
   'Самая быстрорастущая компания по производству авто в России.',
   'Самая быстрорастущая марка автомобилей в России.',
   'Merce является самой быстрорастущей маркой автомобилей в России.',
   'Мерседесе назвали самой быстрорастущей маркой в России.',
   'Самая быстрорастущая марка автомобилей в России.',
   'Мерседесе признали самой быстрой машиной в России.',
   'Самая скоростная российская марка авто "мерседесе".',
   'Названа максимальная скорость самой быстрой иномарки в России.',
   'Названа наиболее быстрорастущая марка легковых автомобилей в России.'],
  'best_candidates': ['Самая быстрорастущая компания по производству авто в России.',
   'Мерседесе назвали самой быстрорастущей маркой в России.',
   'Названа наиболее быстрорастущая марка легковых автомобилей в России.',
   'Самая быстрорастущая марка автомобилей в России.',
   'Merce признали самой быстрой машиной в России.',
   'Merce является самой быстро

In [11]:
para_results = []
for i in tqdm(toxic_inputs):
    results = paraphraser.generate(
        i, n=10, temperature=1,
        top_k=10, top_p=0.9,
        max_length=100, repetition_penalty=1.0,
        threshold=0.7
    )
    para_result = []
    for result in results['results']:
        if result['best_candidates']:
            para_result.append(result['best_candidates'][-1])
        else:
            para_result.append(result['predictions'][0])
    para_results.append(' '.join(para_result))

100%|██████████| 800/800 [35:25<00:00,  2.66s/it]


In [12]:
with open(f'data/output/{model_name}-no-repetition-penalty_dev.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence+'\n' for sentence in para_results])

## HuggingFace version

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from russian_paraphrasers.utils import clean

In [4]:
checkpoint = 'alenusch/rugpt3-paraphraser'
model_name = checkpoint.split('/')[1]

In [5]:
model = GPT2LMHeadModel.from_pretrained(checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint)
model.cuda();

In [20]:
sentence = '<s>Мама мыла раму'
prompt = f'{sentence} === '

In [21]:
encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
encoded_prompt = encoded_prompt.to(model.device)

In [22]:
encoded_prompt

tensor([[   32,    87,    34, 35695, 43068,   351,   597,   225,  6188,    33,
           225]], device='cuda:0')

In [23]:
input_ids = encoded_prompt

In [24]:
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=100,
    temperature=1,
    top_k=10,
    top_p=0.9,
    repetition_penalty=1.5,
    do_sample=True,
    num_return_sequences=10
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [25]:
tokenizer.decode(output_sequences[0])

'<s>Мама мыла раму ===  Мама драила рамы в доме.</s>\n<s>А как у тебя дела сынок === Ну что там малыш мой сладкий даш мне свою руку для рукопожатия а? Ты же не против правда ведь дорогая и всё такое потому папа ты так добр ко всем нам разве нет дорогой</s>\n<s>Я знаю на сколько продвинулась эта работа профессор === Я понимаю насколько сложная операция была проведена вашим'

In [33]:
def paraphrase(
        text,
        model,
        tokenizer,
        max_length=100,
        temperature=1,
        top_k=10,
        top_p=0.9,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=1,
        stop_token='</s>'
):
    prompt = f'<s>{text} === '
    encoded_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(model.device)
    input_ids = encoded_prompt
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=do_sample,
        num_return_sequences=num_return_sequences
    )
    output_text = tokenizer.decode(output_sequences[0])
    output_text = output_text[: output_text.find(stop_token) if stop_token else None]
    output_text = output_text[len(tokenizer.decode(encoded_prompt[0])):]
    output_text = clean(output_text)

    return output_text

In [35]:
para_results = []
for i in tqdm(toxic_inputs):
    para_result = paraphrase(i, model, tokenizer)
    para_results.append(para_result)

100%|██████████| 800/800 [19:33<00:00,  1.47s/it]


In [38]:
with open(f'data/output/{model_name}_dev.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence+'\n' for sentence in para_results])

In [39]:
model_name

'rugpt3-paraphraser'